In [1]:
import os
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
from timm import create_model

In [ ]:

# **Paths**
DATA_DIR = "D:/oral_cancer_detection/data"
MODEL_DIR = "trained_models"
MODEL_PATH = os.path.join(MODEL_DIR, "vit_model.pth")

# **Set device**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# **Ensure Model Directory Exists**
os.makedirs(MODEL_DIR, exist_ok=True)

# **Data Transformations**
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# **Function to Remove Corrupt Images**
def remove_corrupt_images(data_dir):
    corrupt_images = []
    for class_folder in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_folder)
        if not os.path.isdir(class_path):
            continue
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            try:
                with Image.open(img_path) as img:
                    img.verify()
            except (IOError, SyntaxError):
                print(f"\U0001F6A8 Removing corrupt image: {img_path}")
                corrupt_images.append(img_path)
    for img_path in corrupt_images:
        os.remove(img_path)

# **Remove Corrupt Images Before Loading Dataset**
remove_corrupt_images(DATA_DIR)

# **Load Dataset with Transformations**
dataset = datasets.ImageFolder(DATA_DIR, transform=transform)
print(f"Class Mapping: {dataset.class_to_idx}")

# **Split Dataset (80% Train, 20% Validation)**
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# **Data Loaders**
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# **Load ViT Model**
try:
    model = create_model("vit_base_patch16_224", pretrained=True)
except:
    print("Downloading ViT model...")
    os.system("pip install timm")
    from timm import create_model
    model = create_model("vit_base_patch16_224", pretrained=True)

# **Modify Classifier**
model.head = nn.Sequential(
    nn.Linear(model.head.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, 2)
)

model.to(device)

# **Loss and Optimizer**
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.head.parameters(), lr=0.001)

# **Training Loop with Progress Bar**
EPOCHS = 30
for epoch in range(EPOCHS):
    model.train()
    total_loss, correct = 0, 0
    loop = tqdm(train_loader, leave=True, desc=f"Epoch [{epoch+1}/{EPOCHS}]")
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

        loop.set_postfix(loss=total_loss / len(train_loader), acc=100 * correct / len(train_dataset))

# **Save Model**
torch.save(model.state_dict(), MODEL_PATH)
print(f"✅ Training complete. Model saved at {MODEL_PATH}")


Using device: cuda
Class Mapping: {'cancer': 0, 'non_cancer': 1}


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

d:\oral_cancer_detection\venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\msiva\.cache\huggingface\hub\models--timm--vit_base_patch16_224.augreg2_in21k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Epoch [12/30]:  69%|██████▉   | 20/29 [16:27<08:30, 56.67s/it, acc=69.5, loss=0.0198] 